In [1]:
!python -m pip install --upgrade pip

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [3]:
!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

In [4]:
!pip install -q datasets

In [5]:
!pip install pillow

In [11]:
!pip uninstall tesseract

Found existing installation: tesseract 0.1.3
Not uninstalling tesseract at /home/mchou001/.local/lib/python3.7/site-packages, outside environment /home/mchou001/envs/default-tensorflow-gpu-1.15.0
Can't uninstall 'tesseract'. No files were found to uninstall.


In [25]:
!pip install configparser

In [28]:
!pip install pyvoro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyvoro: filename=pyvoro-1.3.2-cp37-cp37m-linux_x86_64.whl size=173645 sha256=42dfeb45da7f62ccbe013a81a6034106488ccb85e33af846e7bb528bcd1d0824
  Stored in directory: /home/mchou001/.cache/pip/wheels/2f/21/18/c570bac5ef4bda4a0c93bd144d01060dd19aef15164488938d
Successfully built pyvoro


In [29]:
import tesseract

ModuleNotFoundError: No module named 'voro'

In [9]:
!pip install pytesseract

In [13]:
from transformers import LayoutLMv2FeatureExtractor, LayoutLMv2Tokenizer, LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from datasets import Dataset
import pandas as pd
import numpy as np
import requests, zipfile, io
import os
from sklearn.preprocessing import LabelEncoder
from PIL import Image, ImageDraw, ImageFont

In [14]:
feature_extractor = LayoutLMv2FeatureExtractor()
tokenizer = LayoutLMv2Tokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased")
processor = LayoutLMv2Processor(feature_extractor, tokenizer)

In [15]:
train = pd.read_csv("train.csv")
train

,filename,labels
0,/home/mchou001/train/1/1.png,Label-TitlePage
1,/home/mchou001/train/1/2.png,Label-Dedication
2,/home/mchou001/train/1/3.png,Label-Abstract
3,/home/mchou001/train/1/4.png,Label-Acknowledgement
4,/home/mchou001/train/1/5.png,Label-TableofContent
...,...,...
5600,/home/mchou001/train/40/48.png,Label-Appendices
5601,/home/mchou001/train/40/49.png,Label-Appendices
5602,/home/mchou001/train/40/50.png,Label-Appendices
5603,/home/mchou001/train/40/51.png,Label-Appendices


In [16]:
y_classes = np.asarray(train['labels'])
le = LabelEncoder()
le.fit(list(y_classes))
train_y = le.transform(list(y_classes))
id2label = dict(zip(train_y, y_classes))

In [17]:
label2id = {k: v for v, k in enumerate(train_y)}
label2id = dict(zip(y_classes, train_y))
label2id

{'Label-TitlePage': 12,
 'Label-Dedication': 4,
 'Label-Abstract': 0,
 'Label-Acknowledgement': 1,
 'Label-TableofContent': 11,
 'Label-ListofFigures': 7,
 'Label-ListofTables': 8,
 'Label-Chapters': 3,
 'Label-Appendices': 2,
 'Label-Other': 9,
 'Label-ReferenceList': 10,
 'Label-DedicationOrAcknowledgement': 5,
 'Label-GeneralAbstract': 6}

In [18]:
train_data = Dataset.from_pandas(train)
train_data

Dataset({
    features: ['filename', 'labels'],
    num_rows: 5605
})

In [19]:
labels = list(set(y_classes))
labels

['Label-Chapters',
 'Label-Acknowledgement',
 'Label-ReferenceList',
 'Label-GeneralAbstract',
 'Label-ListofFigures',
 'Label-ListofTables',
 'Label-TableofContent',
 'Label-Abstract',
 'Label-Other',
 'Label-TitlePage',
 'Label-Appendices',
 'Label-DedicationOrAcknowledgement',
 'Label-Dedication']

In [20]:
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': ClassLabel(num_classes=len(labels), names= labels),
})

def preprocess_data(examples):
    
    images = [Image.open(path).convert("RGB") for path in examples['filename']]
    
    encoded_inputs = processor(images, padding="max_length", truncation=True)
    
    encoded_inputs["labels"] = [label2id[labels] for label in examples["labels"]]
    
    return encoded_inputs

In [21]:
import pytesseract
encoded_dataset = train_data.map(preprocess_data, remove_columns=train_data.column_names, features=features, 
                              batched=True, batch_size=2)

  0%|          | 0/2803 [00:00<?, ?ba/s]

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.